In [22]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff, get_basemaps
import rasterio
from pyproj import Transformer
import numpy as np
import matplotlib.pyplot as plt

In [23]:
def get_bounding_box(geotiff_path):
    with rasterio.open(geotiff_path) as dataset:

        transform = dataset.transform
        width = dataset.width
        height = dataset.height
        
        crs = dataset.crs

        transformer = Transformer.from_crs(crs, "EPSG:4326", always_xy=True)

        top_left = (transform.c, transform.f)
        bottom_right = (transform * (width, height))
        
        min_x, max_y = top_left
        max_x, min_y = bottom_right
        min_lon, max_lat = transformer.transform(min_x, max_y)
        max_lon, min_lat = transformer.transform(max_x, min_y)
        
        # Return bounding box in lat/lon
        return (min_lon, min_lat, max_lon, max_lat)
    


def normalize_to_uint8(input_path, output_path): #samgeo requires uint8 for some reason 

    with rasterio.open(input_path) as src:
        img_data = src.read([2,3,4])
        profile = src.profile

        img_data_norm = np.zeros_like(img_data, dtype=np.uint8)
        for i in range(img_data.shape[0]): 
            img_data_norm[i] = np.interp(img_data[i], (img_data[i].min(), img_data[i].max()), (0, 255)).astype(np.uint8)

        profile.update(dtype=rasterio.uint8, count=3)
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(img_data_norm)

    print(f"Image saved as {output_path}")



In [ ]:
#This is adapted from: https://samgeo.gishub.org/examples/satellite/ , but im not sure how the leafmap module works. 

image = r'/mnt/c/Users/attic/planet_mask/PlanetScope_VarietyCoaslines/20201007_173159_88_2206_3B_AnalyticMS_SR_8b_harmonized_clip.tif'
bbox = get_bounding_box(image)
print(bbox)

m = leafmap.Map(center=[bbox[0], bbox[1]], zoom=19)
m.add_basemap("SATELLITE")
m


In [ ]:
tms_to_geotiff(output=image, bbox=[bbox[0],bbox[1],bbox[2],bbox[3]], zoom=20, source="Satellite", overwrite=True)

In [ ]:
m.layers[-1].visible = False  # turn off the basemap
m.add_raster(image)
m

In [26]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    sam_kwargs=None,
)

In [ ]:

file_path ='/mnt/c/Users/attic/planet_mask/PlanetScope_VarietyCoaslines'
image = r'20201007_173159_88_2206_3B_AnalyticMS_SR_8b_harmonized_clip.tif'
image_path = os.path.join(file_path,image)
output_image = os.path.join('/mnt/c/Users/attic/planet_mask/planet_8bit',image)
normalize_to_uint8(image_path, output_image)

In [31]:
mask = f"segmented_{image}"
sam.generate(
    output_image, mask,foreground=True, mask_multiplier=255
)

In [ ]:
with rasterio.open(mask,'r') as src:
    img = src.read()
plt.figure(figsize=(10,10))
plt.imshow(img[0])
plt.colorbar()
plt.show()